In [ ]:
! pip install evaluate datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer , PretrainedConfig, set_seed
import evaluate
import numpy as np

In [ ]:


from camel_tools.utils.dediac import dediac_ar, dediac_bw
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.charsets import UNICODE_PUNCT_CHARSET, AR_CHARSET, AR_LETTERS_CHARSET
from camel_tools.utils.charmap import CharMapper
from camel_tools.utils.transliterate import Transliterator

import pandas as pd
import numpy as np
import datasets
import os
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from sklearn.metrics import cohen_kappa_score

from transformers import AutoConfig, BertForSequenceClassification, AutoTokenizer, BertTokenizer, PretrainedConfig
from transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch.nn.functional as F
#import numpy as np
#from transformers import Trainer
import torch
#import sys
#import os
#sys.path.append(os.getcwd())
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from src.loss_functions import *


barec_7_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 2,
6: 2,
7: 2,
8: 3,
9: 3,
10: 4,
11: 4,
12: 5,
13: 5,
14: 6,
15: 6,
16: 7,
17: 7,
18: 7,
19: 7
}

barec_5_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 1,
6: 1,
7: 1,
8: 2,
9: 2,
10: 2,
11: 2,
12: 3,
13: 3,
14: 4,
15: 4,
16: 5,
17: 5,
18: 5,
19: 5
}

barec_3_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 1,
6: 1,
7: 1,
8: 1,
9: 1,
10: 1,
11: 1,
12: 2,
13: 2,
14: 3,
15: 3,
16: 3,
17: 3,
18: 3,
19: 3
}

In [ ]:
loss_type = "CE"
d_mat_type = "default" #don't change
n_levels = 13 #don't change

checkpoint = "aubmindlab/bert-base-arabertv02"

data_path = '/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv' # DATA PATH

input_text = "word_sents"

In [ ]:
# save_dir = "/content/drive/MyDrive/MBZ/Thesis/subsets"+checkpoint.split("/")[1]+"_"+input_text+"_"+loss_type+"_"+d_mat_type+"_"+str(n_levels)+"levels"  # SAVE DIRECTORY
# out_xlsx = "/content/drive/MyDrive/MBZ/Thesis/subsets/"+checkpoint.split("/")[1]+"_"+str(n_levels)+"levels"+".xlsx"



In [ ]:
if d_mat_type == "default":
    d_matrix =  [[abs(i-j) for i in range(n_levels)] for j in range(n_levels)]
else:
    d_matrix = [[(abs(i-j)/18)+(abs(barec_7_dict[i+1]-barec_7_dict[j+1])/6)+(abs(barec_5_dict[i+1]-barec_5_dict[j+1])/4)+(abs(barec_3_dict[i+1]-barec_3_dict[j+1])/2) for i in range(19)] for j in range(19)]


In [ ]:
losses_dict = {"CE": Trainer,}
loss_function = losses_dict[loss_type]


In [ ]:
all_df = pd.read_csv(data_path, header=0)
# all_df = all_df[all_df['RL_num_19']>11]



In [ ]:
level_map = {}  # Initialize an empty dictionary for the mapping

for i in range(1, 20):  # Iterate through values 1 to 19
    if i < 12:
        level_map[i] = i  # Map values less than 12 to i
    else:
        level_map[i] = 12  # Map values 12 and above to 12

all_df['RL_num_13'] = all_df['RL_num_19'].map(level_map)  # Apply the mapping to the 'RL_num_19' column and store the result in a new column 'level_map'

In [ ]:
all_df['RL_num_13'].value_counts()

In [ ]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

real_names = {
    input_text: DATA_COLUMN,
    'RL_num_'+str(n_levels): LABEL_COLUMN
}

all_df.rename(columns= real_names, inplace=True)

In [ ]:
minus_mapper = {}
for i in range(n_levels):
  minus_mapper[i+1] = i
all_df = all_df.replace({LABEL_COLUMN: minus_mapper})

In [ ]:
all_df[LABEL_COLUMN].unique()

In [ ]:
len(all_df)

In [ ]:

all_df = all_df.groupby('Split')

all_df = all_df[[DATA_COLUMN, LABEL_COLUMN]]
all_df.columns = [DATA_COLUMN, LABEL_COLUMN]

train_df = all_df.get_group('Train')
#train_df = train_df.head(4565)
dev_df = all_df.get_group('Dev')
test_df = all_df.get_group('Test')
#tune_df = all_df.get_group('Tune')



In [ ]:
len(train_df)

In [ ]:
set_seed(42)

In [ ]:
train = datasets.Dataset.from_pandas(train_df)
dev = datasets.Dataset.from_pandas(dev_df)
test = datasets.Dataset.from_pandas(test_df)
#tune = datasets.Dataset.from_pandas(tune_df)
dataset = load_dataset("labr") #dump loading .. only to match the dataset template from huggingface
dataset['train'] = train
#dataset['tune'] = tune
dataset['dev'] = dev
dataset['test'] = test

In [ ]:
 len(dataset['train'])

In [ ]:
len(dataset['dev'])

In [ ]:
len(dataset['test'])

In [ ]:
train_size = len(dataset['train'])
dev_size = len(dataset['dev'])
test_size = len(dataset['test'])

# Define subsets of the training data
# train_size // 8, train_size // 6, train_size // 4, [train_size // 2,
subset_sizes =  [train_size ]

In [ ]:
def model_init():
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = n_levels, ignore_mismatched_sizes=True)
  model.dist_matix = [[abs(i-j) for i in range(model.num_labels)] for j in range(model.num_labels)]
  for param in model.parameters(): param.data = param.data.contiguous()
  return model

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds,digits=4))
  print(confusion_matrix(p.label_ids,preds))

  #f1_Positive = f1_score(p.label_ids,preds,pos_label=1,average='binary')
  #f1_Negative = f1_score(p.label_ids,preds,pos_label=0,average='binary')
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  #QWK = cohen_kappa_score(p.label_ids, preds, weights='quadratic')
  return {
      #'f1_pos': f1_Positive,
      #'f1_neg': f1_Negative,
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
      #'Quadratic Weighted Kappa': QWK
  }

In [ ]:
# plus_mapper = {}
# for i in range(19):
#   plus_mapper[i] = i+1

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def rank_simple(vector):
    return [i+1 for i in sorted(range(len(vector)), key=vector.__getitem__, reverse=True)]

In [ ]:
####### BAREC levels to SAMER levels
samer_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 2,
    9: 2,
    10: 2,
    11: 2,
    12: 3,
    13: 3,
    14: 4,
    15: 4,
    16: 5,
    17: 5,
    18: 5,
    19: 5
}

###### BAREC levels to BAREC Groups
barec_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 3,
    9: 3,
    10: 4,
    11: 4,
    12: 5,
    13: 5,
    14: 6,
    15: 6,
    16: 6,
    17: 7,
    18: 7,
    19: 7
}


In [ ]:
for subset_size in subset_sizes:
    print('training model with subsize: ', subset_size)
    save_dir = "/content/drive/MyDrive/MBZ/Thesis/bert_13_levels"  # SAVE DIRECTORY
    out_xlsx = "/content/drive/MyDrive/MBZ/Thesis/bert_13_levels/"+str(13)+"levels_"+".xlsx"
    # Create subset of the training data
    train_subset = dataset['train'].select(range(subset_size))
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def tokenize_function(example):
        return tokenizer(example[DATA_COLUMN], truncation=True, max_length=512)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_train_subset = train_subset.map(tokenize_function, batched=True)
    #tokenized_datasets = tokenized_datasets.remove_columns(LABEL_COLUMN)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(save_dir,
                                  evaluation_strategy="epoch",
                                  num_train_epochs=6,
                                  per_device_train_batch_size= 64,
                                  per_device_eval_batch_size=16,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="eval_loss",
                                  greater_is_better=False,
                                  save_strategy="epoch",
                                  #overwrite_output_dir=True,
                                  #save_steps=496,
                                  save_total_limit=1,
                                #   push_to_hub=True,
                                #   hub_token='hub_token',
                                  #learning_rate=lr
                                  )
    trainer = loss_function(model_init=model_init,
                  args = training_args,
                  train_dataset = tokenized_train_subset,
                  eval_dataset = tokenized_datasets['dev'],
                  data_collator=data_collator,
                  tokenizer=tokenizer,
                  compute_metrics = compute_metrics)
    trainer.train()
    trainer.save_model(save_dir)

    preds, labels, metrics = trainer.predict(tokenized_datasets['dev'])

    # probs = {}
    # for i in range(19):
    #   probs[i+1] = []
    # ranks = {}
    # for i in range(19):
    #   ranks[i+1] = []
    texts = []
    original_texts = []
    labels = []
    predictions = []
    # samer_labels = []
    # samer_predictions = []
    # barec_labels = []
    # barec_predictions = []
    # is_equal = []
    # samer_is_equal = []
    # barec_is_equal = []
    # within_one = []
    # within_top_2 = []
    # within_top_3 = []
    # within_top_4 = []
    # rank_of_correct = []
    # diff = []

    for i in range(len(preds)):
      texts.append(list(dev_df['text'])[i])
      # texts.append(list(train_df['text'])[i])
      # original_texts.append(list(original_test['text'])[i])
      labels.append(list(dev_df['label'])[i]+1)
      # labels.append(list(train_df['label'])[i]+1)
      predictions.append(argmax(preds[i])+1)
    #   samer_labels.append(samer_dict[labels[-1]])
    #   barec_labels.append(barec_dict[labels[-1]])
    #   samer_predictions.append(samer_dict[predictions[-1]])
    #   barec_predictions.append(barec_dict[predictions[-1]])
    #   softs = softmax(preds[i])
    #   rank = rank_simple(softs)

    #   if labels[-1] == predictions[-1]:
    #     is_equal.append(1)
    #   else:
    #     is_equal.append(0)

    #   if samer_labels[-1] == samer_predictions[-1]:
    #     samer_is_equal.append(1)
    #   else:
    #     samer_is_equal.append(0)

    #   if barec_labels[-1] == barec_predictions[-1]:
    #     barec_is_equal.append(1)
    #   else:
    #     barec_is_equal.append(0)

    #   if abs(labels[-1] - predictions[-1]) <= 1:
    #     within_one.append(1)
    #   else:
    #     within_one.append(0)

    #   if labels[-1] in rank[:2]:
    #     within_top_2.append(1)
    #   else:
    #     within_top_2.append(0)

    #   if labels[-1] in rank[:3]:
    #     within_top_3.append(1)
    #   else:
    #     within_top_3.append(0)

    #   if labels[-1] in rank[:4]:
    #     within_top_4.append(1)
    #   else:
    #     within_top_4.append(0)

    #   rank_of_correct.append(rank.index(labels[-1])+1)
    #   diff.append(max([labels[-1],predictions[-1]])-min([labels[-1],predictions[-1]]))
    #   for j in range(19):
    #     probs[j+1].append(softs[j])
    #     ranks[j+1].append(rank[j])



    # QWK = cohen_kappa_score(labels, predictions, weights='quadratic')
    # acc = sum(is_equal)/len(is_equal)
    # acc_samer = sum(samer_is_equal)/len(samer_is_equal)
    # acc_barec = sum(barec_is_equal)/len(barec_is_equal)
    # acc_within_one_level = sum(within_one)/len(within_one)
    # acc_top_2 = sum(within_top_2)/len(within_top_2)
    # acc_top_3 = sum(within_top_3)/len(within_top_3)
    # acc_top_4 = sum(within_top_4)/len(within_top_4)
    # avg_rank = sum(rank_of_correct)/len(rank_of_correct)
    # avg_distance = sum(diff)/len(diff)

    # print(f"Accuracy: {acc*100:.4f}")
    # print(f"Accuracy with margin of one level: {acc_within_one_level*100:.4f}")
    # print(f"Accuracy of top 2 choices: {acc_top_2*100:.4f}")
    # print(f"Accuracy of top 3 choices: {acc_top_3*100:.4f}")
    # print(f"Accuracy of top 4 choices: {acc_top_4*100:.4f}")
    # print(f"Accuracy of SAMER levels: {acc_samer*100:.4f}")
    # print(f"Accuracy of BAREC groups: {acc_barec*100:.4f}")
    # print(f"Quadratic Weighted Kappa: {QWK*100:.4f}")
    # print(f"Average rank of correct label: {avg_rank:.4f}")
    # print(f"Average distance between labels and predictions: {avg_distance:.4f}")


    v = {
        #'original text': original_texts,
        'text': texts,
        'label': labels,
        'prediction': predictions,
        # 'is_equal': is_equal,
        # 'samer_label': samer_labels,
        # 'samer_prediction': samer_predictions,
        # 'samer_is_equal': samer_is_equal,
        # 'barec_group_label': barec_labels,
        # 'barec_group_prediction': barec_predictions,
        # 'barec_group_is_equal': barec_is_equal,
        # 'within_one_level': within_one,
        # 'within_top2_ranks': within_top_2,
        # 'within_top3_ranks': within_top_3,
        # 'within_top4_ranks': within_top_4,
        # 'rank_of_correct_label': rank_of_correct,
        # 'diff': diff
    }

    # for i in range(1,20):
    #   s = 'p'+str(i)
    #   v[s] = probs[i]
    # for i in range(1,20):
    #   s = 'rank'+str(i)
    #   v[s] = ranks[i]

    final_df = pd.DataFrame.from_dict(v)
    final_df.to_excel(out_xlsx ,index=False)

In [ ]:
trainer.save_model(save_dir)

preds, labels, metrics = trainer.predict(tokenized_datasets['dev'])

# probs = {}
# for i in range(19):
#   probs[i+1] = []
# ranks = {}
# for i in range(19):
#   ranks[i+1] = []
texts = []
original_texts = []
labels = []
predictions = []
# samer_labels = []
# samer_predictions = []
# barec_labels = []
# barec_predictions = []
# is_equal = []
# samer_is_equal = []
# barec_is_equal = []
# within_one = []
# within_top_2 = []
# within_top_3 = []
# within_top_4 = []
# rank_of_correct = []
# diff = []

for i in range(len(preds)):
  texts.append(list(dev_df['text'])[i])
  # texts.append(list(train_df['text'])[i])
  # original_texts.append(list(original_test['text'])[i])
  labels.append(list(dev_df['label'])[i]+1)
  # labels.append(list(train_df['label'])[i]+1)
  predictions.append(argmax(preds[i])+1)
#   samer_labels.append(samer_dict[labels[-1]])
#   barec_labels.append(barec_dict[labels[-1]])
#   samer_predictions.append(samer_dict[predictions[-1]])
#   barec_predictions.append(barec_dict[predictions[-1]])
#   softs = softmax(preds[i])
#   rank = rank_simple(softs)

#   if labels[-1] == predictions[-1]:
#     is_equal.append(1)
#   else:
#     is_equal.append(0)

#   if samer_labels[-1] == samer_predictions[-1]:
#     samer_is_equal.append(1)
#   else:
#     samer_is_equal.append(0)

#   if barec_labels[-1] == barec_predictions[-1]:
#     barec_is_equal.append(1)
#   else:
#     barec_is_equal.append(0)

#   if abs(labels[-1] - predictions[-1]) <= 1:
#     within_one.append(1)
#   else:
#     within_one.append(0)

#   if labels[-1] in rank[:2]:
#     within_top_2.append(1)
#   else:
#     within_top_2.append(0)

#   if labels[-1] in rank[:3]:
#     within_top_3.append(1)
#   else:
#     within_top_3.append(0)

#   if labels[-1] in rank[:4]:
#     within_top_4.append(1)
#   else:
#     within_top_4.append(0)

#   rank_of_correct.append(rank.index(labels[-1])+1)
#   diff.append(max([labels[-1],predictions[-1]])-min([labels[-1],predictions[-1]]))
#   for j in range(19):
#     probs[j+1].append(softs[j])
#     ranks[j+1].append(rank[j])



# QWK = cohen_kappa_score(labels, predictions, weights='quadratic')
# acc = sum(is_equal)/len(is_equal)
# acc_samer = sum(samer_is_equal)/len(samer_is_equal)
# acc_barec = sum(barec_is_equal)/len(barec_is_equal)
# acc_within_one_level = sum(within_one)/len(within_one)
# acc_top_2 = sum(within_top_2)/len(within_top_2)
# acc_top_3 = sum(within_top_3)/len(within_top_3)
# acc_top_4 = sum(within_top_4)/len(within_top_4)
# avg_rank = sum(rank_of_correct)/len(rank_of_correct)
# avg_distance = sum(diff)/len(diff)

# print(f"Accuracy: {acc*100:.4f}")
# print(f"Accuracy with margin of one level: {acc_within_one_level*100:.4f}")
# print(f"Accuracy of top 2 choices: {acc_top_2*100:.4f}")
# print(f"Accuracy of top 3 choices: {acc_top_3*100:.4f}")
# print(f"Accuracy of top 4 choices: {acc_top_4*100:.4f}")
# print(f"Accuracy of SAMER levels: {acc_samer*100:.4f}")
# print(f"Accuracy of BAREC groups: {acc_barec*100:.4f}")
# print(f"Quadratic Weighted Kappa: {QWK*100:.4f}")
# print(f"Average rank of correct label: {avg_rank:.4f}")
# print(f"Average distance between labels and predictions: {avg_distance:.4f}")


v = {
    #'original text': original_texts,
    'text': texts,
    'label': labels,
    'prediction': predictions,
    # 'is_equal': is_equal,
    # 'samer_label': samer_labels,
    # 'samer_prediction': samer_predictions,
    # 'samer_is_equal': samer_is_equal,
    # 'barec_group_label': barec_labels,
    # 'barec_group_prediction': barec_predictions,
    # 'barec_group_is_equal': barec_is_equal,
    # 'within_one_level': within_one,
    # 'within_top2_ranks': within_top_2,
    # 'within_top3_ranks': within_top_3,
    # 'within_top4_ranks': within_top_4,
    # 'rank_of_correct_label': rank_of_correct,
    # 'diff': diff
}

# for i in range(1,20):
#   s = 'p'+str(i)
#   v[s] = probs[i]
# for i in range(1,20):
#   s = 'rank'+str(i)
#   v[s] = ranks[i]

final_df = pd.DataFrame.from_dict(v)
final_df.to_excel(out_xlsx ,index=False)

Binary -> 11 model , 12 model

In [ ]:
import pandas as pd

In [ ]:
# open the binary file , merge on sentence with the whole data set.
# open /content/drive/MyDrive/MBZ/Thesis/subsets_binary/2levels_8753.xlsx
# binary = pd.read_excel('/content/drive/MyDrive/MBZ/Thesis/subsets_binary/2levels_13130.xlsx')

full_data = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv')
full_data= full_data[full_data['Split'] == 'Test']
# rename word_sents in full data to text
full_data.rename(columns={'word_sents': 'text'}, inplace=True)

# merge on sentence
# full_data = full_data.merge(binary, on='text', how='left')
# create 2 data sets one for 1 and one for 2
# full_data_1 = full_data[full_data['prediction'] == 1.0]
# full_data_2 = full_data[full_data['prediction'] == 2.0]

# predict 1, with the 11s model
model11 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/11levels_26977')

# predict 2, witht the 12s model
model12 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/12levels_25544')

model19 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/19levels_52521')



In [ ]:
# predict using the models
tokenizer11 = AutoTokenizer.from_pretrained('Noorrabie/11levels_26977')

tokenizer12 = AutoTokenizer.from_pretrained('Noorrabie/12levels_25544')

tokenizer19 = AutoTokenizer.from_pretrained('Noorrabie/19levels_52521')



In [ ]:
from transformers import Trainer
from datasets import Dataset

# Batch prediction function
def batch_predict(df, model, tokenizer):
    dataset = Dataset.from_pandas(df[['text']])
    dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    trainer = Trainer(model=model, tokenizer=tokenizer)
    outputs = trainer.predict(dataset)
    preds = outputs.predictions.argmax(axis=1)
    return preds


In [ ]:
data_11= full_data[full_data['RL_num_19']<12]

In [ ]:
data_12 = full_data[full_data['RL_num_19']>=12]

In [ ]:
len(data_11)

In [ ]:
len(preds_19)

In [ ]:

preds_11 = batch_predict(data_11, model11, tokenizer11)
preds_11 = preds_11 + 1
data_11["final_prediction"] = preds_11

# Predict for full_data_2 using 12-class model
preds_12 = batch_predict(data_12, model12, tokenizer12)
preds_12 = preds_12 + 1
data_12["final_prediction"] = preds_12

preds_19 = batch_predict(full_data, model19, tokenizer19)
preds_19 = preds_19 + 1
full_data["final_prediction"] = preds_19

In [ ]:
data_11.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/11_test_predictions.csv", index=False)
data_12.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/12_test_predictions.csv", index=False)
full_data.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/19_test_predictions.csv", index=False)

In [ ]:
# get classification report for full_data_1
from sklearn.metrics import classification_report
print(classification_report(full_data_1['RL_num_19'], full_data_1['final_prediction']))

In [ ]:
# get classification report for full_data_2
print(classification_report(full_data_2['RL_num_19'], full_data_2['final_prediction']))


In [ ]:
# Combine both dataframes
final_predictions_df = pd.concat([full_data_1, full_data_2])

# sort by ID
final_predictions_df = final_predictions_df.sort_values(by='ID')

# Save or analyze
final_predictions_df.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets_11/final_predictions.csv", index=False)


In [ ]:
print(classification_report(final_predictions_df['RL_num_19'], final_predictions_df['final_prediction']))


In [ ]:
final_predictions_df

In [ ]:
# check for duplicate ids in final_predictions_df
len(final_predictions_df[final_predictions_df.duplicated(subset='ID', keep=False)])

In [ ]:
final_predictions_df[final_predictions_df.duplicated(subset='ID', keep=False)]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import Trainer
from transformers import Trainer
from datasets import Dataset
import pandas as pd

full_data = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv')
full_data= full_data[full_data['Split'] == 'Test']
# rename word_sents in full data to text
full_data.rename(columns={'word_sents': 'text'}, inplace=True)

# Load tokenizer and model
model_name = "Noorrabie/bert_content"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Batch prediction function
def batch_predict(df, model, tokenizer):
    dataset = Dataset.from_pandas(df[['text']])
    dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    trainer = Trainer(model=model, tokenizer=tokenizer)
    outputs = trainer.predict(dataset)
    preds = outputs.predictions.argmax(axis=1)
    return preds

preds= batch_predict(full_data, model, tokenizer)
full_data["content_level"] = preds


In [ ]:
full_data